## Experts Answers Data Integration
---

In [1]:
import pandas as pd

In [2]:
excels = ['Drug-disease relations annotations - Medical Expert 1.xlsx',
          'Drug-disease relations annotations - Medical Expert 2.xlsx',
          'Drug-disease relations annotations - Medical Expert 3.xlsx']
sheets = ['Indication_treatment','indication_symptomatic_relief','contraindication', 'effect']
tables = []
for excel in excels:
    for sheet in sheets:
        df = pd.read_excel(excel, sheet_name=sheet)
        df['expert'] = excel[37:53]
        df['sheet'] = sheet
        tables.append(df)

#### Integrate

In [3]:
merged_table = pd.DataFrame()
for table in tables:
    merged_table = merged_table.append(table, ignore_index=True, sort=False)

---
## Gathering experts tables

In [4]:
labels = pd.read_csv('Expert2Relation_Mapping.csv',sep='\t')

In [5]:
merged_table.expert.unique()

array(['Medical Expert 1', 'Medical Expert 2', 'Medical Expert 3'],
      dtype=object)

In [6]:
merged_table = pd.merge(merged_table, labels, 
                        how='left', left_on ='medical_expert', right_on ='text') #add the correct labels

In [7]:
merged_table.relation.unique()

array([nan, 'Indication: Symptomatic Relief', 'Indication: Treatment',
       'Contraindication', 'Inconsistent', 'Effect', 'Duplicate', 'IDK'],
      dtype=object)

In [8]:
subset = merged_table[['disease_name','drug_name','context','expert','relation']]\
    .rename(columns={'relation':'expert_label'})\
    .dropna()
subset['value'] = 1
subset.drop_duplicates(keep='first', inplace=True)

## Create an expert consensus table

In [9]:
expert_table = pd.pivot_table(
    subset,
    values='value',
    index=['disease_name','drug_name','context'],
    columns=['expert_label'],
    aggfunc='sum',
    fill_value=0
).reset_index().drop(columns=['Inconsistent','Duplicate'])

In [10]:
expert_table['expert_answers'] = expert_table.sum(axis=1)
expert_table['Contraindication'] = expert_table['Contraindication']/expert_table['expert_answers']
expert_table['Effect'] = expert_table['Effect']/expert_table['expert_answers']
expert_table['IDK'] = expert_table['IDK']/expert_table['expert_answers']
expert_table['Indication: Symptomatic Relief'] = expert_table['Indication: Symptomatic Relief']/expert_table['expert_answers']
expert_table['Indication: Treatment'] = expert_table['Indication: Treatment']/expert_table['expert_answers']
expert_table['inter_agree_experts'] = expert_table.drop(columns='expert_answers').max(axis=1)

In [11]:
label_list = list(labels.relation.unique())
label_list.remove('Inconsistent'); label_list.remove('Duplicate')
consensus = []
for i in range(len(expert_table)):
    if expert_table.loc[i].inter_agree_experts > .51:
        for r in label_list:
            if expert_table.loc[i].inter_agree_experts == expert_table.loc[i][str(r)]:
                consensus.append(str(r))
    else:
        consensus.append('No consensus')

expert_table['expert_consensus'] = consensus

In [12]:
expert_table.head()

expert_label,disease_name,drug_name,context,Contraindication,Effect,IDK,Indication: Symptomatic Relief,Indication: Treatment,expert_answers,inter_agree_experts,expert_consensus
0,ACNE,TAZAROTENE,Retinoids may cause fetal harm when administer...,0.0,0.0,0.0,0.000000,1.000000,2,1.000000,Indication: Treatment
1,ACUTE CERVICITIS,LEVONORGESTREL,['The use of LILETTA is contraindicated when o...,1.0,0.0,0.0,0.000000,0.000000,3,1.000000,Contraindication
2,ACUTE INTERMITTENT PORPHYRIA,Hemin,PANHEMATIN is a hemin for injection indicated ...,0.0,0.0,0.0,0.666667,0.333333,3,0.666667,Indication: Symptomatic Relief
3,ACUTE LYMPHOBLASTIC LEUKEMIA,Vincristine,Marqibo is a vinca alkaloid indicated for the ...,0.0,0.0,0.0,0.000000,1.000000,2,1.000000,Indication: Treatment
4,ACUTE LYMPHOCYTIC LEUKEMIA,CYTARABINE,Cytarabine Injection in combination with other...,0.0,0.0,0.0,0.000000,1.000000,2,1.000000,Indication: Treatment


In [13]:
expert_table.shape

(368, 11)

In [14]:
expert_table.to_csv('expert_table.csv',index=False)